# Import Libraries & Tools

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import os

# Import & Organize Data

The data came from various files found on the [Open Doors](https://opendoorsdata.org/data/us-study-abroad/) website. I downloaded 6 `.xlsx` files initially, and opened them in Google Sheets to view them. I was planning on doing 100% of the cleaning in this project, but after looking at the format of the files I decided to pre-clean them. Here is a list of what I did:

* Removed blank/empty rows
* Removed redundant or unnecessary header rows
* Removed data before 2010 - some files did not have data reported for every year
* Converted file to `.csv` format

Aside from that, I have kept the data intact as it was originally prepared. There is still a significant amount of cleaning that will be necessary before being able to combine all of our data and analyze it.

In [2]:
#get path to project directory
DATA_DIR = os.path.join(
    os.path.dirname(os.path.realpath("__file__")), 'data'
)

#individual data files
DURATION_FILE    = os.path.join(DATA_DIR, 'durations.csv')
DESTINATION_FILE = os.path.join(DATA_DIR, 'destinations.csv')
FIELD_FILE       = os.path.join(DATA_DIR, 'fields_of_study.csv')
PHD_FILE         = os.path.join(DATA_DIR, 'rank_phd.csv')
MASTER_FILE      = os.path.join(DATA_DIR, 'rank_master.csv')
BACHELOR_FILE    = os.path.join(DATA_DIR, 'rank_bachelor.csv')
PROFILE_FILE     = os.path.join(DATA_DIR, 'student_profiles.csv')
TOTAL_FILE       = os.path.join(DATA_DIR, 'study_abroad_totals.csv')

files = [DURATION_FILE, DESTINATION_FILE, FIELD_FILE, PHD_FILE,
        MASTER_FILE, BACHELOR_FILE, PROFILE_FILE, TOTAL_FILE]

data = {}
pattern = r"(\w+)(?=\.)" #gets file name without file extension

#loops through each .csv file and reads it in, while setting the file name as a key
for f in files:
    d = pd.read_csv(f)
    key_name = re.findall(pattern, f)[0]
    data[key_name] = d #each key has one .csv as its value

# Data Exploration

I have included a small description of each of the above listed files:

* **Durations** - percentages of U.S. students that studied abroad for various durations
* **Destinations** - shows most popular destination countries
* **Fields of Study** - percentages of study abroad students in various fields of study
* **Institutions** - rank of schools based on academic level (doctorate, master's, bachelor's)
* **Student Profiles** - shows characteristics of study abroad students (academic level, gender, ethnicity)
* **Study Abroad Totals** - leading schools awarding credit for studying abroad

## Helper Functions


I have created a function to nicely display all columns containing missing/null values, the percentage of values that are missing/null, and the datatype. We'll begin by identifying which files have missing/null values, if any. Note that this function only detects if a value is `NaN`, not if the value contains something like a '`-`'.  

In addition, I've also created a couple minor functions to rename/edit column headers, and visualize missing values.

In [3]:
#nifty function to help detect missing & null values
def missing_zero_values_table(df, name='Data'):
    mis_val = df.isnull().sum()
    
    if mis_val.sum() == 0:
        print(f'{name.capitalize()} has no missing/null values!')
    else:
        #calculates and organizes all zero/missing data
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mz_table = pd.concat([zero_val, mis_val], axis=1)
        mz_table = mz_table.rename(columns = {0 : 'Zero Values', 1 : 'Missing Values'})
        mz_table['% Zero'] = 100 * zero_val / len(df)
        mz_table['% Missing'] = 100 * mis_val / len(df)
        mz_table['Zero/Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Zero/Missing'] = 100 * mz_table['Zero/Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        
        #reorders columns
        col_order = ['Zero Values', '% Zero', 'Missing Values', '% Missing',
                     'Zero/Missing Values', '% Zero/Missing', 'Data Type']
        mz_table = mz_table.reindex(columns=col_order)
        
        #sorts by chosen col and prints messages (defaults to missing %)
        sort_by_col = 3
        mz_table = mz_table[mz_table.iloc[:,sort_by_col] != 0].sort_values(
            col_order[sort_by_col], ascending=False).round(1)
        
        print(f'{name.capitalize()} shape: {df.shape}. Columns w/ missing values: {mz_table.shape[0]}')
        # print(f'There are {mz_table.shape[0]} columns that have missing values.\n')
        return mz_table
    
#function to convert string numbers to numeric
def make_num(data):
    if type(data) == str:
        if ',' in data:
            data = data.replace(',','')
        elif data == '-' or '.' in data: # NaN values in original .csvs
            pass
        else:
            return data
    data = pd.to_numeric(data, errors='coerce')
    return data

#make year column header consistent
def make_year(data):
    return data.replace('/','-')

#heatmap hack to show missing values
def missing_vis(df):
    sns.heatmap(df.isnull(), 
                yticklabels=False, 
                cbar=False, 
                cmap='viridis')
    plt.show()
    
file_names = ['destinations', 'durations', 'fields_of_study', 'rank_phd',
              'rank_master', 'rank_bachelor', 'student_profiles', 'study_abroad_totals']
    
for f in file_names:
    missing_zero_values_table(data[f], f)

Destinations shape: (26, 49). Columns w/ missing values: 9
Durations has no missing/null values!
Fields_of_study has no missing/null values!
Rank_phd shape: (21, 58). Columns w/ missing values: 9
Rank_master shape: (21, 58). Columns w/ missing values: 9
Rank_bachelor shape: (21, 58). Columns w/ missing values: 9
Student_profiles has no missing/null values!
Study_abroad_totals shape: (26, 98). Columns w/ missing values: 49


Because they have missing values, the following files will need some form of imputation:

* destinations
* rank_phd
* rank_master
* rank_bachelor
* study_abroad_totals

Since the `rank` files are all similar, we can create a function to help speed up the process. For now, we'll focus on cleaning the files one by one and figuring out the imputation methods as we encounter them.

## Data Cleaning - Destinations

In [4]:
missing_zero_values_table(data['destinations'], 'destinations')

Destinations shape: (26, 49). Columns w/ missing values: 9


,Zero Values,% Zero,Missing Values,% Missing,Zero/Missing Values,% Zero/Missing,Data Type
Unnamed: 4,0,0.0,26,100.0,26,100.0,float64
Unnamed: 9,0,0.0,26,100.0,26,100.0,float64
Unnamed: 14,0,0.0,26,100.0,26,100.0,float64
Unnamed: 19,0,0.0,26,100.0,26,100.0,float64
Unnamed: 24,0,0.0,26,100.0,26,100.0,float64
Unnamed: 29,0,0.0,26,100.0,26,100.0,float64
Unnamed: 34,0,0.0,26,100.0,26,100.0,float64
Unnamed: 39,0,0.0,26,100.0,26,100.0,float64
Unnamed: 44,0,0.0,26,100.0,26,100.0,float64


In [5]:
data['destinations'] = (
    data['destinations'].dropna(axis=1) #removes Unnamed columns mentioned above
    .rename(columns=data['destinations'].iloc[0]) #new columns headers
    .drop(0) #removes columns we just turned into headers
    .reset_index(drop=True)
)

#separate into individual years
year = ['2010-11', '2011-12', '2012-13', '2013-14', '2014-15',
         '2015-16', '2016-17', '2017-18', '2018-19', '2019-20']
years = {}

for i, y in enumerate(year):
    years[y] = data['destinations'].iloc[:,(i*4):(i*4 +4)]
    years[y].insert(0, 'Year', y) #adds year column so we can differentiate

#create dataframe for our combined destination-years
destinations = pd.DataFrame(columns=['Year', 'Rank', 'Destination', 'Students', '% of Total'])

#concat each year
for y in year:
    destinations = pd.concat([destinations, years[y]])

#convert to make sure number columns are numeric
destinations = destinations.applymap(make_num)
destinations

,Year,Rank,Destination,Students,% of Total
0,2010-11,1,United Kingdom,33182,12.1
1,2010-11,2,Italy,30361,11.1
2,2010-11,3,Spain,25965,9.5
3,2010-11,4,France,17019,6.2
4,2010-11,5,China,14596,5.3
...,...,...,...,...,...
20,2019-20,21,Ecuador,1787,1.1
21,2019-20,22,India,1736,1.1
22,2019-20,23,Austria,1405,0.9
23,2019-20,24,Chile,1332,0.8


We now have a clean dataset for our destinations file. We can perform some aggregation on it to get some basic statistics regarding the study abroad program in the U.S.

In [6]:
by_country_year = destinations.groupby(['Destination', 'Year']).agg(sum)
by_country_year

Students  % of Total
Destination    Year                         
Argentina      2010-11      4589         1.7
               2011-12      4763         1.7
               2012-13      4549         1.6
               2013-14      4301         1.4
               2014-15      3708         1.2
...                          ...         ...
United Kingdom 2015-16     39140        12.0
               2016-17     39851        12.0
               2017-18     39403        11.5
               2018-19     39358        11.3
               2019-20     19147        11.8

[250 rows x 2 columns]

## PUT SOME AGGREGATIONS AND VISUALIZATIONS HERE?

## Data Cleaning - Durations

In [7]:
missing_zero_values_table(data['durations'], 'durations')

labels = {
    '8 Weeks or Less During Academic Year': '< 8 weeks',
    '    Two to eight weeks': '2-8 weeks',
    '    Fewer than two weeks': '< 2 weeks',
    '    Summer: More than eight weeks': 'Summer: > 8 weeks',
    '    Summer: Two to eight weeks': 'Summer: 2-8 weeks',
    '    Summer: Fewer than two weeks': 'Summer: < 2 weeks',
    'One Quarter': '1 Quarter',
    'Two Quarters': '2 Quarters',
    'One Semester': '1 Semester',
    'Total': 'Total # of Students'
}

data['durations'] = (
    data['durations'].rename(columns={'Duration of Study': 'Duration of Study (%)'})
    .rename(columns=make_year)
    .replace(labels)
    .set_index('Duration of Study (%)')
    .applymap(make_num)
    .apply(lambda row: row.fillna(round(row.mean(),1)), axis=1) #imputate mean
)

durations = data['durations']
durations

Durations has no missing/null values!


,2010-11,2011-12,2012-13,2013-14,2014-15,2015-16,2016-17,2017-18,2018-19,2019-20
Duration of Study (%),,,,,,,,,,
< 8 weeks,13.3,14.4,15.3,16.5,16.7,17.4,18.8,19.0,19.3,15.9
2-8 weeks,5.0,6.5,6.9,6.6,6.5,6.6,6.8,7.3,6.9,5.7
< 2 weeks,8.3,7.9,8.4,9.9,10.2,10.8,12.0,11.7,12.4,10.2
January Term,7.1,7.0,7.1,7.5,7.4,7.4,7.1,7.0,6.9,13.9
Summer Term,37.7,37.1,37.8,38.1,39.0,38.0,38.5,38.5,38.6,0.9
Summer: > 8 weeks,2.3,2.3,2.3,2.3,2.7,2.6,2.9,2.9,2.6,0.1
Summer: 2-8 weeks,34.4,33.4,33.7,33.5,30.9,30.4,30.5,29.9,29.6,0.6
Summer: < 2 weeks,3.3,3.7,4.1,4.6,5.4,5.0,5.1,5.7,6.4,0.2
1 Quarter,3.0,2.5,2.4,2.4,2.2,2.3,2.2,2.4,1.8,2.8


In [8]:
dur_num_students = durations.iloc[-1,0:].sum().astype(int)
des_num_students = destinations['Students'].sum()

#function to check student counts between files
def print_diff(f1, f2, d1, d2):
    print(f"{f'# Students ({f1}.csv): {d1}':>40}")
    print(f"{f'# Students ({f2}.csv): {d2}':>40}")
    print(f"{'Difference:':>32} {f'{d1 - d2}':>7}")

print_diff('durations', 'destinations', dur_num_students, des_num_students)

     # Students (durations.csv): 2974167
  # Students (destinations.csv): 2226890
                     Difference:  747277


In the original `.xlsx` file, there are two notes at the bottom:

* Note: Historical data may not always sum to totals.
* Note: Regional names and data reflect Open Doors 2021. This may not match data in historical publications.

This may help explain the difference in total number of students between the two files, though admittedly that feels like a very large number. We're off by roughly *75,000* students for each year (on average). We will ignore it for the time being.

## Data Cleaning - Fields of Study

In [9]:
data['fields_of_study'] = (
    data['fields_of_study'].rename(columns=make_year)
    .rename(columns={'Unnamed: 0': 'Field of Study (%)'})
    .drop(0)
    .set_index('Field of Study (%)')
    .applymap(make_num)
    .apply(lambda row: row.fillna(round(row.mean(),1)), axis=1) #imputate mean
)

fields = data['fields_of_study']
fields

,2010-11,2011-12,2012-13,2013-14,2014-15,2015-16,2016-17,2017-18,2018-19,2019-20
Field of Study (%),,,,,,,,,,
Physical or Life Sciences,7.9,8.6,8.8,8.0,8.1,8.1,8.0,7.8,8.1,7.4
Health Professions,5.3,5.7,6.4,6.0,6.3,7.1,7.1,6.9,7.1,6.0
Engineering,3.5,3.9,4.1,4.6,5.0,5.1,5.3,5.2,5.5,4.3
Math or Computer Science,1.8,1.7,1.9,2.1,2.2,2.4,2.8,2.9,3.2,3.8
Agriculture,1.3,1.2,1.3,1.9,2.3,2.5,2.5,2.7,2.9,3.0
Business & Management,20.5,20.5,20.4,19.6,20.1,20.9,20.7,20.8,20.7,20.8
Social Sciences**,22.9,22.4,22.1,18.7,17.3,17.1,17.2,17.1,17.0,19.2
Foreign Language and International Studies***,7.4,7.4,7.4,7.8,7.7,7.4,7.3,7.1,6.9,7.8
Fine and Applied Arts,8.2,7.8,7.8,7.0,6.9,6.9,6.3,6.8,6.9,6.9


* Note: All broad field of study categories are based on Open Doors 2021 classifications, which may not match data in historical publications.
* Note: The fields of study used in the Open Doors 2021 report are from Classification of Instructional Programs, 2020 Edition, published by the National Center for Education Statistics (NCES) of the U.S. Department of Education.
* Note: Prior to Open Doors 2021 The fields of study used were from Classification of Instructional Programs, 2010 Edition, published by the National Center for Education Statistics (NCES) of the U.S. Department of Education.
* ** Beginning in 2013/14, changes were made in the classification of fields of study reported in the Open Doors U.S. Study Abroad Survey. Figures reported from 2013/14 onward are not entirely comparable to prior years.	
* *** Beginning in 2013/14, Communications and Journalism  and Legal Studies & Law Enforcement were reported separately; and Foreign Language was merged with International Studies.

## Data Cleaning - Rank_phd

In [10]:
example = (
    data['rank_phd'].dropna(axis=1) #removes Unnamed columns mentioned above
    .rename(columns=data['rank_phd'].iloc[0]) #new columns headers
    .drop(0) #removes columns we just turned into headers
    .reset_index(drop=True)
)

For the `rank` files, there's an interesting mistake with the columns. This file has 10 years worth of data, and each year has 5 columns of information. This should give us 50 total columns.

In [11]:
print(example.shape)
example.columns

(20, 49)


Index(['Rank', 'Institutions', 'City', 'State', 'Total Study Abroad Students',
       'Rank', 'Institutions', 'City', 'State', 'Total Study Abroad Students',
       'Rank', 'Institutions', 'City', 'State', 'Total Study Abroad Students',
       'Rank', 'Institutions', 'City', 'State', 'Total Study Abroad Students',
       'Rank', 'Institutions', 'City', 'State', 'Total Study Abroad Students',
       'Rank', 'Institutions', 'City', 'State', 'Total Study Abroad Students',
       'Rank', 'Institutions', 'City', 'State', 'Total Study Abroad Students',
       'Rank', 'Institutions', 'City', 'State', 'Total Study Abroad Students',
       'Rank', 'Institutions', 'City', 'State', 'Total Study Abroad Students',
       'Institutions', 'City', 'State', 'Total Study Abroad Students'],
      dtype='object')

However if you look at the shape, we're actually missing a column. The last line shows it's missing a `Rank` column. We'll have to insert one manually. Because the `rank` files were all part of the same original `.xlsx` file, they all have this same discrepancy. We can go ahead and create a function that will perform all the processing, so that way we don't have to repeat ourselves.

In [12]:
def process_rank(df):
    df = (
        df.dropna(axis=1) #removes Unnamed columns mentioned above
        .rename(columns=df.iloc[0]) #new columns headers
        .drop(0) #removes columns we just turned into headers
        .reset_index(drop=True)
    )
    
    #replace that missing column
    df.insert(45, 'Rank', df.iloc[:,0], True)

    #separate into individual years
    year_dict = {}

    for i, y in enumerate(year):
        year_dict[y] = df.iloc[:,(i*5):(i*5 +5)]
        year_dict[y].insert(0, 'Year', y) #adds year column so we can differentiate

    #create dataframe for our combined
    new_df = pd.DataFrame()

    #concat each year
    for y in year:
        new_df = pd.concat([new_df, year_dict[y]])

    cols = {'Institutions': 'Institution',
            'Total Study Abroad Students': 'Total # of Students'}

    new_df.rename(columns=cols, inplace=True)

    #convert to make sure number columns are numeric
    new_df = new_df.applymap(make_num)
    return new_df

In [13]:
phd = process_rank(data['rank_phd'])
master = process_rank(data['rank_master'])
bachelor = process_rank(data['rank_bachelor'])

Now that we have the `rank` files cleaned up, we can perform aggregate functions on them to get a better understanding of the relationship between study abroad students and the types of degrees they were pursuing.

In [14]:
students_per_year = phd.groupby(['Institution', 'Year']).agg(sum)
students_per_year

Total # of Students
Institution                       Year                        
Arizona State University - Tempe  2014-15                 2102
                                  2015-16                 2222
                                  2016-17                 2414
                                  2017-18                 2567
                                  2018-19                 2651
...                                                        ...
University of Wisconsin - Madison 2015-16                 2244
                                  2016-17                 2276
                                  2017-18                 2410
                                  2018-19                 2547
                                  2019-20                 1602

[200 rows x 1 columns]

In [15]:
students_by_citystate = phd.groupby(['Year', 'State', 'City']).agg(sum)
students_by_citystate

Total # of Students
Year    State City                                
2010-11 CA    Los Angeles                     4791
        FL    Gainesville                     2075
        GA    Athens                          2079
        IL    Champaign                       1907
        IN    Bloomington                     2203
...                                            ...
2019-20 PA    University Park                 1189
        SC    Columbia                        1253
        TX    College Station                 1504
        WA    Seattle                         1467
        WI    Madison                         1602

[191 rows x 1 columns]

## Data Cleaning - Student Profiles

In [16]:
data['student_profiles']

labels = {
    '     Associate\'s Students': 'Associate\'s Students',
    '     Freshman': 'Freshman',
    '     Sophomore': 'Sophomore',
    '     Junior': 'Junior',
    '     Senior': 'Senior',
    '     Bachelor\'s, Unspecified': 'Bachelor\'s, Unspecified',
    '     Master\'s Students': 'Master\'s Students',
    '     Graduate, Professional': 'Graudate, Professional',
    '     Doctoral Students': 'Doctoral Students',
    '     Graduate, Unspecified': 'Graduate, Unspecified',
    'Asian, Native Hawaiian or Other Pacific Islander': 'Asian, Native Hawaiian or Pacific Islander',
    'TOTAL': 'Total # of Students'
}

data['student_profiles'] = (
    data['student_profiles'].rename(columns={'Unnamed: 0': 'Characteristic'})
    .rename(columns=make_year)
    .replace(labels)
    .drop([0, 7])
    .set_index('Characteristic')
    .applymap(make_num)
)

profiles = data['student_profiles']
academic = profiles.iloc[:11,:].copy()
ethnicity = profiles.iloc[11:,:].copy()

In [24]:
academic

,2010-11,2011-12,2012-13,2013-14,2014-15,2015-16,2016-17,2017-18,2018-19,2019-20
Characteristic,,,,,,,,,,
Associate's Students,0.2,1.1,1.1,1.7,1.8,1.7,1.7,1.7,1.9,0.8
Freshman,3.3,3.3,3.8,3.9,3.9,3.6,4.0,4.2,4.1,2.7
Sophomore,12.6,13.0,13.7,13.1,13.1,12.7,13.2,12.8,13.2,12.3
Junior,35.8,36.0,34.7,33.9,33.1,32.9,33.0,33.0,33.4,42.7
Senior,23.4,24.4,24.7,25.3,26.4,27.7,27.4,28.2,29.4,27.0
"Bachelor's, Unspecified",10.3,8.4,8.4,9.1,9.3,9.1,8.6,7.8,6.2,5.1
Master's Students,8.5,8.3,8.4,7.6,7.4,7.0,7.3,7.8,7.4,5.4
"Graudate, Professional",2.3,2.6,2.3,2.0,1.9,2.1,2.0,0.8,0.7,1.6
Doctoral Students,0.6,0.7,0.8,0.7,0.7,0.7,0.7,1.8,1.7,0.6


In [18]:
ethnicity

,2010-11,2011-12,2012-13,2013-14,2014-15,2015-16,2016-17,2017-18,2018-19,2019-20
Characteristic,,,,,,,,,,
Women,64.4,64.8,65.3,65.3,66.6,66.5,67.3,67.0,67.3,67.4
Men,35.6,35.2,34.7,34.7,33.4,33.5,32.7,33.0,32.7,32.6
White,77.8,76.4,76.3,74.3,72.9,71.6,70.8,70.0,68.7,70.0
Hispanic or Latino(a),6.9,7.6,7.6,8.3,8.8,9.7,10.2,10.6,10.9,10.6
"Asian, Native Hawaiian or Pacific Islander",7.9,7.7,7.3,7.7,8.1,8.4,8.2,8.4,8.9,8.6
Black or African-American,4.8,5.3,5.3,5.6,5.6,5.9,6.1,6.1,6.4,5.5
Multiracial,2.1,2.5,3.0,3.6,4.1,3.9,4.3,4.4,4.7,4.8
American Indian or Alaska Native,0.5,0.5,0.5,0.5,0.5,0.5,0.4,0.5,0.4,0.5
Total # of Students,273996.0,283332.0,289408.0,304467.0,313415.0,325339.0,332727.0,341751.0,347099.0,162633.0
